<a href="https://colab.research.google.com/github/Sakthikeerthika-T/Emotion-recognition-eeg-/blob/main/feature_extarction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# connect to drive :

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Import modules :

In [ ]:
# import packages, libraries
from scipy.io import loadmat
import numpy as np
import matplotlib.pyplot as plt
import scipy.io

!pip install mne
import mne

from scipy.signal import welch
import pywt
import scipy.io as sio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 49.3 MB/s eta 0:00:00


# load dataset :

In [ ]:
file_path = '/content/drive/MyDrive/dataset folder/s32.mat'

In [ ]:
dataset = scipy.io.loadmat(file_path)   #loading dataset
variables = dataset.keys()
print(variables)    #print data_dictionary keys

dict_keys(['__header__', '__version__', '__globals__', 'labels', 'data'])


In [ ]:
data = dataset['data']
label = dataset['labels']
print(data.shape)
print(label.shape)
type(data)
type(label)

(40, 40, 8064)
(40, 4)


numpy.ndarray

In [ ]:
eeg_data = data[:, :-8, :]
type(eeg_data)
eeg_data.shape

(40, 32, 8064)

# FEATURE EXTRACTION :

In [ ]:
def compute_activity(data):
    activity = np.sum(np.abs(data), axis=2) / data.shape[2]         #axis = 2 means number of samples
    return activity

def compute_mobility(data):
    mobility = np.sqrt(np.var(data, axis=2))
    return mobility

def compute_complexity(data):
    gradients = np.gradient(data, axis=2)
    complexity = np.divide(np.sqrt(np.var(gradients, axis=2)), compute_mobility(data))
    return complexity

In [ ]:
def compute_sum_first_diff(data):
    first_diff = np.diff(data, axis=2)
    sum_first_diff = np.sum(first_diff, axis=2)
    return sum_first_diff


def compute_sum_normalized_first_diff(data):
    first_diff = np.diff(data, axis=2)
    normalized_first_diff = first_diff / np.std(first_diff, axis=2, keepdims=True)
    sum_normalized_first_diff = np.sum(normalized_first_diff, axis=2)
    return sum_normalized_first_diff


In [ ]:
def band_power_extraction(data, fs = 128):
    # Define the frequency bands
    frequency_bands = {
        'delta': (0.5, 4),
        'theta': (4, 8),
        'alpha': (8, 13),
        'beta': (13, 30),
        'gamma': (30, 45)
    }

    # Compute the power spectral density for each channel and trial
    num_trials, num_channels, num_samples = data.shape
    power_array = np.zeros((num_trials, num_channels, len(frequency_bands)))

    for trial_idx in range(num_trials):
        for channel_idx in range(num_channels):
            channel_data = data[trial_idx, channel_idx, :]
            for band_idx, (f_low, f_high) in enumerate(frequency_bands.values()):
                f, Pxx = welch(channel_data, fs=fs, nperseg=256, noverlap=128)
                power_array[trial_idx, channel_idx, band_idx] = np.sum(Pxx[(f >= f_low) & (f <= f_high)])

    return power_array


In [ ]:
band_powers = band_power_extraction(eeg_data)

In [ ]:
array1 = band_powers[..., 0]
array2 = band_powers[..., 1]
array3 = band_powers[..., 2]
array4 = band_powers[..., 3]
array5 = band_powers[..., 4]

In [ ]:
#import numpy as np

def compute_higher_order_crossing(data, threshold=0):
    binary_sequence = np.sign(data - threshold)
    crossings = np.sum(np.abs(np.diff(binary_sequence, axis=2)), axis=2) / 2
    return crossings

hoc_result = compute_higher_order_crossing(eeg_data)

# Print the shape of the resulting HOC array
print(hoc_result.shape)


(40, 32)


In [ ]:
def calculate_dwt_rms(data):
    trials, channels, samples = data.shape
    rms_values = np.zeros((trials, channels))

    for trial in range(trials):
        for channel in range(channels):
            dwt_coeffs = pywt.dwt(data[trial, channel, :], 'db1')  # Perform DWT using 'db1' wavelet
            dwt_rms = np.sqrt(np.mean(np.square(dwt_coeffs)))
            rms_values[trial, channel] = dwt_rms

    return rms_values



def calculate_dwt_ree(data):
    trials, channels, samples = data.shape
    ree_values = np.zeros((trials, channels))

    for trial in range(trials):
        for channel in range(channels):
            dwt_coeffs = pywt.dwt(data[trial, channel, :], 'db1')  # Perform DWT using 'db1' wavelet
            energy = np.sum(np.square(dwt_coeffs))
            ree = energy / (samples * np.var(data[trial, channel, :]))
            ree_values[trial, channel] = ree

    return ree_values



In [ ]:
def concatenate_features_(data):
    activity = compute_activity(data)
    mobility = compute_mobility(data)
    complexity = compute_complexity(data)
    sum_first_diff = compute_sum_first_diff(data)
    sum_normalized_first_diff = compute_sum_normalized_first_diff(data)
    higher_order_crossing = compute_higher_order_crossing(data, threshold=0)
    dwt_ree = calculate_dwt_ree(data)
    dwt_rms =calculate_dwt_rms(data)


    print("Shape of activity:", activity.shape)
    print("Shape of mobility:", mobility.shape)
    print("Shape of complexity:", complexity.shape)
    print("Shape of sum_first_diff:", sum_first_diff.shape)
    print("Shape of sum_normalized_first_diff:", sum_normalized_first_diff.shape)
    print("Shape of higher_order_crossing:", higher_order_crossing.shape)
    print("shape of dwt_rms:",dwt_rms.shape)
    print("shape of dwt_ree:",dwt_ree.shape)

    # Perform the concatenation
    features = np.concatenate((activity, mobility, complexity, sum_first_diff,
                               sum_normalized_first_diff, higher_order_crossing,
                               array1, array2, array3, array4, array5,
                               dwt_rms, dwt_ree), axis=-1)

    return features


In [ ]:
final = concatenate_features_(eeg_data)
print("\nfinal_features_shape",final.shape)

Shape of activity: (40, 32)
Shape of mobility: (40, 32)
Shape of complexity: (40, 32)
Shape of sum_first_diff: (40, 32)
Shape of sum_normalized_first_diff: (40, 32)
Shape of higher_order_crossing: (40, 32)
shape of dwt_rms: (40, 32)
shape of dwt_ree: (40, 32)

final_features_shape (40, 416)


# LABELS :

In [ ]:
label.shape

(40, 4)

In [ ]:
valance,ars,dom,lh = label[:, 0] , label[:,1],label[:,2],label[:,3]
print(valance.shape, ars.shape ,dom.shape, lh.shape)

valance_2D , ars_2d , dom_2d , lh_2d = valance.reshape((40, 1)),ars.reshape(40,1),dom.reshape(40,1),lh.reshape(40,1)


(40,) (40,) (40,) (40,)


# valance :

In [ ]:
# Define the thresholds for the discrete labels
average = np.mean(valance_2D )
print("average valance is : " , average,"\n")

# Define the labels for each range
label_low = "Low"
label_high = "High"

# Function to convert continuous label to discrete label
def convert_label(value):
    if value < average:
        return label_low
    else:
        return label_high

# Iterate over each label and convert it to the discrete label
discrete_labels = [convert_label(label) for label in valance_2D ]

# Print the converted labels for first 5 viedos
for i in range(5):
    print(f"Original label: {valance_2D [i]}, Converted label: {discrete_labels[i]}")

d_valance= np.array(discrete_labels)                     # shape------(40,)

#discrete_labels_array = np.reshape(discrete_labels_array, (-1, 1))   # shape ------------ (40,1)

print("\n shape of discrete_labels array ",d_valance.shape)
print("\n",d_valance)


average valance is :  5.3125 

Original label: [8.13], Converted label: High
Original label: [4.99], Converted label: Low
Original label: [8.05], Converted label: High
Original label: [6.96], Converted label: High
Original label: [7.15], Converted label: High

 shape of discrete_labels array  (40,)

 ['High' 'Low' 'High' 'High' 'High' 'High' 'Low' 'High' 'High' 'Low' 'High'
 'High' 'High' 'High' 'Low' 'Low' 'High' 'High' 'High' 'High' 'Low' 'Low'
 'Low' 'Low' 'Low' 'Low' 'High' 'High' 'Low' 'Low' 'Low' 'Low' 'Low' 'Low'
 'Low' 'Low' 'Low' 'Low' 'Low' 'High']


# ARS

In [ ]:
# Define the thresholds for the discrete labels
average = np.mean(ars_2d)
print("average valance is : " , average,"\n")

# Define the labels for each range
label_low = "Low"
label_high = "High"

# Function to convert continuous label to discrete label
def convert_label(value):
    if value < average:
        return label_low
    else:
        return label_high

# Iterate over each label and convert it to the discrete label
discrete_labels = [convert_label(label) for label in ars_2d]

# Print the converted labels for first 5 viedos
for i in range(5):
    print(f"Original label: {ars_2d [i]}, Converted label: {discrete_labels[i]}")

d_ars= np.array(discrete_labels)                     # shape------(40,)

#discrete_labels_array = np.reshape(discrete_labels_array, (-1, 1))   # shape ------------ (40,1)

print("\n shape of discrete_labels array ",d_ars.shape)
print("\n",d_ars)


average valance is :  5.54425 

Original label: [4.83], Converted label: Low
Original label: [2.99], Converted label: Low
Original label: [7.09], Converted label: High
Original label: [5.14], Converted label: Low
Original label: [5.94], Converted label: High

 shape of discrete_labels array  (40,)

 ['Low' 'Low' 'High' 'Low' 'High' 'Low' 'Low' 'High' 'Low' 'High' 'High'
 'Low' 'Low' 'High' 'Low' 'High' 'Low' 'High' 'High' 'Low' 'High' 'High'
 'High' 'High' 'High' 'Low' 'High' 'Low' 'High' 'High' 'High' 'High' 'Low'
 'High' 'High' 'High' 'High' 'High' 'High' 'Low']


# DOMINANCE :

In [ ]:
# Define the thresholds for the discrete labels
average = np.mean(dom_2d)
print("average valance is : " , average,"\n")

# Define the labels for each range
label_low = "Low"
label_high = "High"

# Function to convert continuous label to discrete label
def convert_label(value):
    if value < average:
        return label_low
    else:
        return label_high

# Iterate over each label and convert it to the discrete label
discrete_labels = [convert_label(label) for label in dom_2d]

# Print the converted labels for first 5 viedos
for i in range(5):
    print(f"Original label: {dom_2d[i]}, Converted label: {discrete_labels[i]}")

d_dom= np.array(discrete_labels)                     # shape------(40,)

#discrete_labels_array = np.reshape(discrete_labels_array, (-1, 1))   # shape ------------ (40,1)

print("\n shape of discrete_labels array ",d_dom.shape)
print("\n",d_dom)


average valance is :  7.792500000000001 

Original label: [9.], Converted label: High
Original label: [7.96], Converted label: High
Original label: [8.86], Converted label: High
Original label: [9.], Converted label: High
Original label: [9.], Converted label: High

 shape of discrete_labels array  (40,)

 ['High' 'High' 'High' 'High' 'High' 'High' 'High' 'High' 'High' 'High'
 'High' 'High' 'High' 'High' 'High' 'Low' 'High' 'High' 'High' 'High'
 'Low' 'High' 'Low' 'Low' 'High' 'High' 'High' 'High' 'High' 'Low' 'Low'
 'Low' 'Low' 'Low' 'Low' 'Low' 'Low' 'Low' 'Low' 'High']


# liklehood :

In [ ]:
# Define the thresholds for the discrete labels
average = np.mean(lh_2d )
print("average valance is : " , average,"\n")

# Define the labels for each range
label_low = "Low"
label_high = "High"

# Function to convert continuous label to discrete label
def convert_label(value):
    if value < average:
        return label_low
    else:
        return label_high

# Iterate over each label and convert it to the discrete label
discrete_labels = [convert_label(label) for label in lh_2d]

# Print the converted labels for first 5 viedos
for i in range(5):
    print(f"Original label: {lh_2d [i]}, Converted label: {discrete_labels[i]}")

d_lh = np.array(discrete_labels)                     # shape------(40,)

#discrete_labels_array = np.reshape(discrete_labels_array, (-1, 1))   # shape ------------ (40,1)

print("\n shape of discrete_labels array ",d_lh.shape)
print("\n",d_lh)


average valance is :  4.5595 

Original label: [4.87], Converted label: High
Original label: [1.01], Converted label: Low
Original label: [7.08], Converted label: High
Original label: [6.14], Converted label: High
Original label: [4.81], Converted label: High

 shape of discrete_labels array  (40,)

 ['High' 'Low' 'High' 'High' 'High' 'Low' 'Low' 'High' 'High' 'Low' 'High'
 'High' 'High' 'High' 'Low' 'High' 'Low' 'High' 'High' 'High' 'Low' 'High'
 'High' 'High' 'High' 'Low' 'High' 'High' 'Low' 'High' 'Low' 'High' 'Low'
 'Low' 'Low' 'Low' 'Low' 'Low' 'Low' 'Low']


# SAVE FEATURES ,LABELS AS FILE :

In [ ]:
save_path= '/content/drive/MyDrive/dataset folder/deap_32.mat'
sio.savemat(save_path, {  'data' : final ,  'labels_v': d_valance , 'labels_a': d_ars ,'labels_d' :d_dom ,'labels_l' :d_lh})